# Using TransformConfig with Long Text

In this example, we demonstrate how to leverage the Transform flow in handling long text that exceeds the token limitation of the ChatGPT API. There are three TransformConfigs for OpenAI that you can add the parameter `auto_split_long_text`
- TransformOpenAIConfig
- TransformForGenerationOpenAIGPT3p5Config
- <del>TransformForClusteringOpenAIGPT4Config</del>

We use the text from [War and Peace](https://github.com/mmcky/nyu-econ-370/blob/master/notebooks/data/book-war-and-peace.txt) as our example.

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

Next, you will need a valid [OpenAI API key](https://platform.openai.com/api-keys) to run the code. Once you have the key, set it as the environment variable `OPENAI_API_KEY` within a `.env` file in the root directory of this repository. For more details, see this [instruction](https://github.com/CambioML/uniflow/tree/main#api-keys)

Finally, we store the "War and Peace" text file in the `data/raw_input` directory as "book-war-and-peace.txt". You can download the file from [here](https://github.com/mmcky/nyu-econ-370/blob/master/notebooks/data/book-war-and-peace.txt).

### Update system path

In [148]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install helper packages

In [ ]:
!{sys.executable} -m pip install -q python-dotenv

In [ ]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5")
enc = tiktoken.get_encoding("cl100k_base")
print(enc)
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:

<Encoding 'cl100k_base'>


### Import Dependency

In [ ]:
from dotenv import load_dotenv
from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformForGenerationOpenAIGPT3p5Config
from uniflow.op.model.model_config import OpenAIModelConfig
from uniflow.op.prompt import Context, PromptTemplate

load_dotenv()

True

### Generate Context object with long text

In [ ]:
def extract_contexts(file_path, ranges):
    # Read the entire file content
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Initialize the list for holding contexts
    contexts = []

    # Extract specified ranges
    for start, end in ranges:
        # Adjusting end index to fit within the file length if necessary
        end = min(end, len(content))
        context_text = content[start:end]
        contexts.append(Context(context=context_text))

    return contexts

# Define the ranges for the contexts
ranges = [(100, 92000), (92000+12600, 92000+13200)]

# Specify the file path
file_path = './data/raw_input/book-war-and-peace.txt'

# Extract contexts
contexts = extract_contexts(file_path, ranges)

for i, context in enumerate(contexts):
    preview_text = context.context[:50] + "..." if len(context.context) > 50 else context.context
    print(f"---\nContext {i+1} (Preview): '{preview_text}'\n---")

---
Context 1 (Preview): 'arn you, if you don't tell me that this means war,...'
---
---
Context 2 (Preview): 'nly she jumped up onto a tub to be higher than he,...'
---


### Prepare sample prompts

In [ ]:
guided_prompt = PromptTemplate(
    instruction="""Generate one question and its corresponding answer based on the last context in the last
    example. Follow the format of the examples below to include context, question, and answer in the response""",
    few_shot_prompt=[
        Context(
            context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
            question="Who published A Mathematical Theory of Communication in 1948?",
            answer="Claude E. Shannon.",
        ),
        Context(
            context="""The Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantum for InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements;""",
            question="What does the Compute & Networking segment include?",
            answer="""The Compute & Networking segment includes Data Center accelerated computing platforms, end-to-end networking platforms (Quantum for InfiniBand and Spectrum for Ethernet), the NVIDIA DRIVE automated-driving platform, and automotive development agreements.""",
        ),
])

### Use LLM to generate data

In this example, we will use the [OpenAIModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L17)'s default LLM to generate questions and answers.

Here, we pass in our `guided_prompt` to the `OpenAIConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

We also want to get the response in the `json` format instead of the `text` default, so we set the `response_format` to `json_object`.

Please note that we include the `auto_split_long_text` parameter in the transform configuration. This ensures that if a `Context` object contains text exceeding the specified token length limit, it will automatically be split into multiple `Context` objects. Each of these objects will contain text segments that adhere to the limit, ready for submission to the OpenAI API.

In [ ]:
config = TransformForGenerationOpenAIGPT3p5Config(
    prompt_template=guided_prompt,
    model_config=OpenAIModelConfig(),
    auto_split_long_text=True
)

client = TransformClient(config)

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [ ]:
output = client.run(contexts)

from pprint import pprint
pprint(output)

Chunk text (first 40 chars): 'arn you, if you don't tell me that this ' has token length: 4031
Chunk text (first 40 chars): 'Anna Pavlovna's alarm was justified, for' has token length: 3888
Chunk text (first 40 chars): 'Influence in society, however, is a capi' has token length: 4068
Chunk text (first 40 chars): '"Princesse, au revoir," cried he, stumbl' has token length: 3967
Chunk text (first 40 chars): 'It was past one o'clock when Pierre left' has token length: 3984
Chunk text (first 40 chars): '"Dear Countess, what an age... She has b' has token length: 2056


100%|██████████| 7/7 [00:06<00:00,  1.07it/s]

[{'output': [{'error': 'No errors.',
              'response': ['question: Who was the well-known Anna Pavlovna '
                           "Scherer's maid of honor and favorite of the "
                           'Empress Marya Fedorovna?\n'
                           "answer: The well-known Anna Pavlovna Scherer's "
                           'maid of honor and favorite of the Empress Marya '
                           'Fedorovna was Prince Vasili Kuragin.']}],
  'root': <uniflow.node.Node object at 0x125da0550>},
 {'output': [{'error': 'No errors.',
              'response': ['question: Which character in the story is '
                           'described as "very lovely"?\n'
                           'answer: Princess Helene.']}],
  'root': <uniflow.node.Node object at 0x12629d760>},
 {'output': [{'error': 'No errors.',
              'response': ['question: Who left the hall after taking leave of '
                           'Anna Pavlovna, having managed to speak to Lise '
   

### Display the output

In [ ]:
from IPython.display import display, HTML

# Define a function to wrap text in HTML tags with style
def format_html_question_answer(question, answer):
    return f"""
    <div style="margin-bottom: 20px;">
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
            <b>Question:</b> {question}
        </div>
        <div style="background-color: #d9edf7; padding: 10px; border-radius: 5px; margin-top: 5px;">
            <b>Answer:</b> {answer}
        </div>
    </div>
    """

html_output = ""
for o in output:
    response = o['output'][0]['response'][0]
    # Split the response based on the first occurrence of '\nanswer'
    split_index = response.find('\nanswer')
    question = response[:split_index].replace('question: ', '')
    answer = response[split_index:].replace('\nanswer: ', '')
    html_output += format_html_question_answer(question, answer)

# Display the formatted HTML
display(HTML(html_output))

## Appendix

- When splitter using token length for this case (Token length for each context: 4096): 100%|██████████| 7/7 [00:09<00:00,  1.33s/it]

- When splitter using char length for this case (Char length for each context: 4096): 100%|██████████| 25/25 [00:27<00:00,  1.11s/it]

(We are using the splitter based on token length for config)


## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example/model#examples)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>